In [31]:
import numpy as np
from math import comb
from dataclasses import dataclass
from typing import Optional, Dict, Any, Tuple, List

from scipy.optimize import linprog


@dataclass
class LP1Result:
    ok: bool
    status: int
    message: str
    s: float
    n: int
    h: int
    C: float
    t: Optional[float] = None
    f: Optional[np.ndarray] = None  # length n, f[0]=f_1
    raw: Optional[Dict[str, Any]] = None


def _attack_value(n: int, h: int, C: float, s: float, f: np.ndarray, d: int) -> float:
    """
    Evaluate the LHS of 'attack delivers d' at a given (f,s).
    LHS(d) = sum_{r=0..h} binom(h,r) s^{h-r}(1-s)^r * f_{d+h-r},
    with convention f_0 = C.
    """
    val = 0.0
    for r in range(h + 1):
        coef = comb(h, r) * (s ** (h - r)) * ((1.0 - s) ** r)
        idx = d + h - r  # 0..n
        if idx == 0:
            val += coef * C
        else:
            val += coef * f[idx - 1]
    return float(val)


def _attack_row(n: int, h: int, C: float, s: float, d: int) -> Tuple[np.ndarray, float]:
    """
    Build one 'attack delivers d' inequality in A_ub x <= b form
    where x = [f_1..f_n, t].
    """
    row = np.zeros(n + 1, dtype=float)
    const = 0.0

    for r in range(h + 1):
        coef = comb(h, r) * (s ** (h - r)) * ((1.0 - s) ** r)
        idx = d + h - r  # this is the f-index (0..n)

        if idx == 0:
            const += coef * C
        else:
            if 1 <= idx <= n:
                row[idx - 1] += coef
            else:
                raise ValueError(f"Attack constraint references f_{idx} outside [0..n]. "
                                 f"Got idx={idx} for d={d}, h={h}, n={n}.")

    # (sum) - t <= -const
    row[-1] = -1.0
    b = -const
    return row, b


def _equilibrium_row(n: int, s: float) -> np.ndarray:
    """
    Equality:
        sum_{i=1..n} comb(n-1, n-i) s^{i-1} (1-s)^{n-i} * (f_i / i) = 1
    """
    row = np.zeros(n + 1, dtype=float)  # last entry (t) stays 0
    for i in range(1, n + 1):
        j = n - i
        coef = comb(n - 1, j) * (s ** (i - 1)) * ((1.0 - s) ** j) / i
        row[i - 1] = coef
    return row


def _ir_value(n: int, s: float, f: np.ndarray) -> float:
    """
    Evaluate RHS of IR inequality:
        RHS = sum_{i=1..n} comb(n,i) s^i (1-s)^{n-i} f_i
    and LHS = n s.
    """
    rhs = 0.0
    for i in range(1, n + 1):
        rhs += comb(n, i) * (s ** i) * ((1.0 - s) ** (n - i)) * f[i - 1]
    return float(rhs)


def _ir_row(n: int, s: float) -> Tuple[np.ndarray, float]:
    """
    IR:
        n s <= sum_i coef_i f_i
    -> -sum_i coef_i f_i <= -n s
    """
    row = np.zeros(n + 1, dtype=float)  # t coefficient 0
    for i in range(1, n + 1):
        coef = comb(n, i) * (s ** i) * ((1.0 - s) ** (n - i))
        row[i - 1] = -coef
    b = -n * s
    return row, b


def solve_lp1_for_s(n: int, h: int, C: float, s: float, *, highs: bool = True) -> LP1Result:
    if not (0 <= s <= 1):
        raise ValueError("s must be in [0,1].")
    if not (0 <= h <= n):
        raise ValueError("Require 0 <= h <= n.")

    # Decision vector x = [f_1..f_n, t]
    c = np.zeros(n + 1, dtype=float)
    c[-1] = 1.0  # minimize t

    A_ub_rows: List[np.ndarray] = []
    b_ub: List[float] = []

    # Attack constraints for d = 0..n-h
    for d in range(0, n - h + 1):
        row, b = _attack_row(n=n, h=h, C=C, s=s, d=d)
        A_ub_rows.append(row)
        b_ub.append(b)

    # IR constraint
    row_ir, b_ir = _ir_row(n=n, s=s)
    A_ub_rows.append(row_ir)
    b_ub.append(b_ir)

    A_ub = np.vstack(A_ub_rows)
    b_ub_arr = np.array(b_ub, dtype=float)

    # Equilibrium equality
    A_eq = _equilibrium_row(n=n, s=s).reshape(1, -1)
    b_eq = np.array([1.0], dtype=float)

    # Bounds: f_i >= 0; t >= 0
    bounds = [(0.0, None)] * (n + 1)

    method = "highs" if highs else "interior-point"
    res = linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub_arr,
        A_eq=A_eq,
        b_eq=b_eq,
        bounds=bounds,
        method=method,
    )

    if res.success:
        f = np.array(res.x[:n], dtype=float)
        t = float(res.x[-1])
        return LP1Result(
            ok=True,
            status=res.status,
            message=res.message,
            s=float(s),
            n=n,
            h=h,
            C=float(C),
            t=t,
            f=f,
            raw={"fun": float(res.fun), "nit": getattr(res, "nit", None)},
        )
    else:
        return LP1Result(
            ok=False,
            status=res.status,
            message=res.message,
            s=float(s),
            n=n,
            h=h,
            C=float(C),
            raw={"fun": getattr(res, "fun", None), "nit": getattr(res, "nit", None)},
        )


def print_constraint_values_at_opt(res: LP1Result, *, tol: float = 1e-9) -> None:
    """
    Pretty-print attack constraint LHS values and IR slack at the optimal solution.
    """
    if not res.ok or res.f is None or res.t is None:
        print("No optimal solution to print (LP infeasible or not solved).")
        return

    n, h, C, s = res.n, res.h, res.C, res.s
    f, t = res.f, res.t

    print(f"\n=== Constraint values at optimum (n={n}, h={h}, C={C}, s={s:.6f}) ===")
    print(f"t* = {t:.12g}")
    print(f"f* = {np.array2string(f, precision=10, floatmode='fixed')}")

    # Attack constraints
    print("\nAttack constraints (LHS should be <= t):")
    worst = -np.inf
    worst_d = None
    for d in range(0, n - h + 1):
        lhs = _attack_value(n=n, h=h, C=C, s=s, f=f, d=d)
        slack = t - lhs
        worst = max(worst, lhs)
        if worst == lhs:
            worst_d = d
        active = abs(slack) <= max(1.0, abs(t)) * tol
        print(
            f"  d={d:2d}  LHS={lhs:.12g}   slack(t-LHS)={slack:.12g}"
            + ("   [ACTIVE]" if active else "")
        )
    print(f"  max_d LHS = {worst:.12g} (attained at d={worst_d})")

    # Equilibrium equality check
    eq_lhs = float(_equilibrium_row(n=n, s=s)[:n] @ f)  # t coefficient is 0
    print("\nEquilibrium equality (LHS should equal 1):")
    print(f"  LHS={eq_lhs:.12g}  residual(LHS-1)={eq_lhs - 1.0:.12g}")

    # IR constraint
    ir_rhs = _ir_value(n=n, s=s, f=f)
    ir_lhs = n * s
    ir_slack = ir_rhs - ir_lhs
    print("\nIR constraint (RHS should be >= n*s):")
    print(f"  n*s = {ir_lhs:.12g}")
    print(f"  RHS = {ir_rhs:.12g}")
    print(f"  slack(RHS - n*s) = {ir_slack:.12g}" + ("   [TIGHT]" if abs(ir_slack) <= max(1.0, abs(ir_rhs)) * tol else ""))


def sweep_best_s(
    n: int,
    h: int,
    C: float,
    *,
    grid: int = 401,
    s_min: float = 0.0,
    s_max: float = 1.0,
    drop_endpoints: bool = False,
) -> Tuple[Optional[LP1Result], List[LP1Result]]:
    if grid < 2:
        raise ValueError("grid must be >= 2")

    ss = np.linspace(s_min, s_max, grid)
    if drop_endpoints:
        ss = ss[1:-1]

    all_results: List[LP1Result] = []
    best: Optional[LP1Result] = None

    for s in ss:
        r = solve_lp1_for_s(n=n, h=h, C=C, s=float(s))
        all_results.append(r)
        if r.ok:
            if best is None or (r.t is not None and r.t < best.t):
                best = r

    return best, all_results

In [104]:
# Example usage
n = 3
h = 1
C = 7.

best, results = sweep_best_s(n=n, h=h, C=C, grid=4001, drop_endpoints=False)

feasible = [r for r in results if r.ok]
print(f"Feasible: {len(feasible)}/{len(results)}")

if best is None:
    print("No feasible s found on the grid.")
else:
    print(f"\nBEST on grid: n={n}, h={h}, C={C}")
    print(f"  s = {best.s:.6f}")
    print(f"  t = {best.t:.12g}")
    print(f"  f = {np.array2string(best.f, precision=3, floatmode='fixed')}")

    # NEW: print constraint values at optimum
    print_constraint_values_at_opt(best, tol=1e-9)


Feasible: 4001/4001

BEST on grid: n=3, h=1, C=7.0
  s = 0.676250
  t = 2.26628486914
  f = [5.156e-05 3.351e+00 1.747e+00]

=== Constraint values at optimum (n=3, h=1, C=7.0, s=0.676250) ===
t* = 2.26628486914
f* = [5.1562500000e-05 3.3512283561e+00 1.7468756952e+00]

Attack constraints (LHS should be <= t):
  d= 0  LHS=2.26628486914   slack(t-LHS)=0   [ACTIVE]
  d= 1  LHS=2.26628486914   slack(t-LHS)=0   [ACTIVE]
  d= 2  LHS=2.26628486914   slack(t-LHS)=0   [ACTIVE]
  max_d LHS = 2.26628486914 (attained at d=2)

Equilibrium equality (LHS should equal 1):
  LHS=1  residual(LHS-1)=-2.22044604925e-16

IR constraint (RHS should be >= n*s):
  n*s = 2.02875
  RHS = 2.02875
  slack(RHS - n*s) = -4.4408920985e-16   [TIGHT]
